# 26. Tabularディープラーニング - Neural Networks for Tabular Data

## 概要
テーブルデータに特化したディープラーニング手法を学びます。PyTorchで構築したニューラルネットワークとSHAPによる解釈可能性を実装し、GBDTとの比較を行います。

## 学習目標
- Tabularデータでのディープラーニングの特徴を理解できる
- PyTorchでTabular用ニューラルネットワークを実装できる
- SHAPを使った解釈可能性を理解できる
- GBDTとの比較ができる
- 実務でTabular DLを適用できる判断ができる

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import lightgbm as lgb
import xgboost as xgb

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# SHAP for interpretability
import shap

import warnings
warnings.filterwarnings('ignore')

# 設定
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)
torch.manual_seed(42)

# MacBook対応：CPUを使用
device = torch.device('cpu')

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

## 1. なぜTabularデータにディープラーニング？

### Tabularデータの特徴

**Tabularデータ（テーブルデータ）**は、行と列で構成された構造化データです。

### 従来の常識

長年、Tabularデータでは**GBDT（Gradient Boosting Decision Trees）が最強**とされてきました：

- ✅ XGBoost、LightGBM、CatBoost
- ✅ Kaggleコンペで圧倒的な勝率
- ✅ 特徴量エンジニアリング不要
- ✅ ハイパーパラメータ調整が比較的容易

### ディープラーニングの課題

一方、従来のニューラルネットワークはTabularデータで苦戦：

- ❌ GBDTより性能が低い
- ❌ 大量のデータが必要
- ❌ 特徴量の前処理が必須（正規化など）
- ❌ ハイパーパラメータ調整が難しい

### 新世代のTabular Deep Learning

近年、Tabularデータ専用に設計されたDLモデルが登場：

1. **TabNet** (Google Research, 2019)
   - 解釈可能性を持つニューラルネットワーク
   - Sequential Attention機構

2. **NODE** (Neural Oblivious Decision Ensembles, 2019)
   - 決定木とNNのハイブリッド

3. **FT-Transformer** (Feature Tokenizer Transformer, 2021)
   - 各特徴量をトークン化

本ノートブックでは、**PyTorchで構築した最適化されたニューラルネットワーク**と**SHAP**を使用して、Tabularデータでのディープラーニングを実装します。

### いつTabular DLを使うべきか

| 条件 | 推奨 |
|------|------|
| データサイズ < 10k | GBDT |
| データサイズ > 100k | Tabular DL検討 |
| 解釈可能性が重要 | GBDT + SHAP または NN + SHAP |
| 高速推論が必要 | GBDT |
| GPUが使える | Tabular DL |
| カテゴリカル変数多い | CatBoost |

## 2. データの準備

In [ ]:
# データセットの読み込み（乳がん診断データ）
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

print(f"データサイズ: {X.shape}")
print(f"特徴量数: {X.shape[1]}")
print(f"サンプル数: {X.shape[0]}")
print(f"\nターゲット分布:")
print(y.value_counts())
print(f"\n特徴量の例:")
print(X.head())

In [ ]:
# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# さらに検証データを分割
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"訓練データ: {X_train.shape}")
print(f"検証データ: {X_val.shape}")
print(f"テストデータ: {X_test.shape}")

# スケーリング（ディープラーニングには必須）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("\n⚠️ 重要:")
print("- ディープラーニングでは特徴量のスケーリングが必須")
print("- 訓練データでfitし、検証・テストデータはtransformのみ")
print("- GBDTはスケーリング不要")

## 3. PyTorchでTabularニューラルネットワークを構築

### Tabular用ニューラルネットワークの設計

**最適なアーキテクチャの特徴：**

1. **Batch Normalization**
   - 各層の入力を正規化
   - 学習の安定化と高速化

2. **Dropout**
   - 過学習を防ぐ
   - 汎化性能の向上

3. **ReLU Activation**
   - 勾配消失問題を回避
   - 計算効率が良い

4. **適切な層の深さと幅**
   - Tabularデータには深すぎないネットワークが有効
   - 通常2-4層で十分

### アーキテクチャ図

```
Input (30 features)
    ↓
Linear(30 → 128)
    ↓
BatchNorm → ReLU → Dropout(0.3)
    ↓
Linear(128 → 64)
    ↓
BatchNorm → ReLU → Dropout(0.3)
    ↓
Linear(64 → 32)
    ↓
BatchNorm → ReLU → Dropout(0.2)
    ↓
Linear(32 → 2)
    ↓
Output (2 classes)
```

In [ ]:
# Tabular用ニューラルネットワークの定義
class TabularNN(nn.Module):
    def __init__(self, input_dim, hidden_dims=[128, 64, 32], output_dim=2, dropout_rates=[0.3, 0.3, 0.2]):
        super(TabularNN, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        # 隠れ層の構築
        for i, hidden_dim in enumerate(hidden_dims):
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.ReLU())
            if i < len(dropout_rates):
                layers.append(nn.Dropout(dropout_rates[i]))
            prev_dim = hidden_dim
        
        # 出力層
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# モデルの初期化
input_dim = X_train_scaled.shape[1]
model = TabularNN(input_dim=input_dim, hidden_dims=[128, 64, 32], output_dim=2)
model = model.to(device)

print("モデルアーキテクチャ:")
print(model)
print(f"\nパラメータ数: {sum(p.numel() for p in model.parameters()):,}")

### 3.1 学習の準備

In [ ]:
# データローダーの準備
def create_dataloader(X, y, batch_size=32, shuffle=True):
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.LongTensor(y.values if isinstance(y, pd.Series) else y)
    dataset = TensorDataset(X_tensor, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_loader = create_dataloader(X_train_scaled, y_train, batch_size=32, shuffle=True)
val_loader = create_dataloader(X_val_scaled, y_val, batch_size=32, shuffle=False)
test_loader = create_dataloader(X_test_scaled, y_test, batch_size=32, shuffle=False)

# 損失関数とオプティマイザ
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

print("学習の準備完了")
print(f"- 損失関数: CrossEntropyLoss")
print(f"- オプティマイザ: Adam (lr=0.001)")
print(f"- スケジューラ: ReduceLROnPlateau")
print(f"- バッチサイズ: 32")

### 3.2 学習ループの実装

In [ ]:
# 学習関数
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += y_batch.size(0)
        correct += predicted.eq(y_batch).sum().item()
    
    return total_loss / len(loader), correct / total

# 評価関数
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += y_batch.size(0)
            correct += predicted.eq(y_batch).sum().item()
            
            probs = torch.softmax(outputs, dim=1)[:, 1]
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    
    auc = roc_auc_score(all_labels, all_probs)
    return total_loss / len(loader), correct / total, auc

print("学習関数の準備完了")

In [ ]:
# 学習の実行
num_epochs = 100
best_val_auc = 0
patience = 15
patience_counter = 0

history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'val_auc': []
}

print("学習開始...\n")

for epoch in range(num_epochs):
    # 訓練
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 検証
    val_loss, val_acc, val_auc = evaluate(model, val_loader, criterion, device)
    
    # スケジューラの更新
    scheduler.step(val_loss)
    
    # 履歴の保存
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_auc'].append(val_auc)
    
    # Early stopping
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        best_epoch = epoch
        patience_counter = 0
        # ベストモデルの保存
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}")
        print(f"  Best Val AUC: {best_val_auc:.4f} (Epoch {best_epoch+1})")
    
    if patience_counter >= patience:
        print(f"\nEarly stopping at epoch {epoch+1}")
        break

# ベストモデルをロード
model.load_state_dict(best_model_state)

print(f"\n学習完了")
print(f"最良エポック: {best_epoch+1}")
print(f"最良検証AUC: {best_val_auc:.4f}")

### 3.3 学習曲線の可視化

In [ ]:
# 学習曲線の可視化
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss曲線
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].axvline(best_epoch, color='red', linestyle='--', 
                label=f'Best Epoch ({best_epoch+1})', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Loss', fontsize=11)
axes[0].set_title('Training and Validation Loss', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy曲線
axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['val_acc'], label='Validation Accuracy', linewidth=2)
axes[1].axvline(best_epoch, color='red', linestyle='--', 
                label=f'Best Epoch ({best_epoch+1})', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('Accuracy', fontsize=11)
axes[1].set_title('Training and Validation Accuracy', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

# AUC曲線
axes[2].plot(history['val_auc'], label='Validation AUC', linewidth=2, color='orange')
axes[2].axvline(best_epoch, color='red', linestyle='--', 
                label=f'Best Epoch ({best_epoch+1})', linewidth=2)
axes[2].set_xlabel('Epoch', fontsize=11)
axes[2].set_ylabel('AUC', fontsize=11)
axes[2].set_title('Validation AUC', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 観察:")
print(f"- 最良エポック: {best_epoch+1}")
print(f"- Early stoppingにより過学習を防止")
print(f"- 検証AUCは {best_val_auc:.4f} に到達")

### 3.4 テストデータでの評価

In [ ]:
# テストデータで評価
model.eval()
y_pred_nn = []
y_proba_nn = []

with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        _, predicted = outputs.max(1)
        probs = torch.softmax(outputs, dim=1)[:, 1]
        
        y_pred_nn.extend(predicted.cpu().numpy())
        y_proba_nn.extend(probs.cpu().numpy())

y_pred_nn = np.array(y_pred_nn)
y_proba_nn = np.array(y_proba_nn)

# 評価
accuracy_nn = accuracy_score(y_test, y_pred_nn)
auc_nn = roc_auc_score(y_test, y_proba_nn)

print("=" * 60)
print("Neural Networkの性能（テストデータ）")
print("=" * 60)
print(f"Accuracy: {accuracy_nn:.4f}")
print(f"ROC-AUC:  {auc_nn:.4f}")
print("\n混同行列:")
print(confusion_matrix(y_test, y_pred_nn))
print("\n詳細レポート:")
print(classification_report(y_test, y_pred_nn, 
                           target_names=['Malignant', 'Benign']))

## 4. SHAP による解釈可能性

### SHAPとは

**SHAP (SHapley Additive exPlanations)** は、ゲーム理論に基づく機械学習モデルの解釈手法です。

### SHAPの特徴

1. **モデル非依存**
   - あらゆる機械学習モデルに適用可能
   - ニューラルネットワーク、GBDT、線形モデルなど

2. **理論的基盤**
   - Shapley値（ゲーム理論）に基づく
   - 数学的に正当化された重要度

3. **グローバルとローカルの解釈**
   - グローバル：全体的な特徴量重要度
   - ローカル：個別予測の説明

4. **可視化**
   - Summary plot、Force plot、Dependence plotなど
   - 直感的な理解が可能

In [ ]:
# SHAPの準備: モデルをラップ
def model_predict(X):
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X).to(device)
        outputs = model(X_tensor)
        probs = torch.softmax(outputs, dim=1)
        return probs.cpu().numpy()

# SHAP Explainerの作成（サンプル数を制限して高速化）
background = shap.sample(X_train_scaled, 100)  # 背景データ
explainer = shap.KernelExplainer(model_predict, background)

# テストデータの一部でSHAP値を計算
test_sample = X_test_scaled[:50]  # 最初の50サンプル
shap_values = explainer.shap_values(test_sample, nsamples=100)

print("SHAP値の計算完了")
print(f"SHAP値の形状: {np.array(shap_values).shape}")

### 4.1 グローバル特徴量重要度

In [ ]:
# Summary plot: グローバル特徴量重要度
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values[1], test_sample, feature_names=X.columns, show=False)
plt.title('SHAP Summary Plot - Global Feature Importance', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 SHAP Summary Plot:")
print("- 縦軸: 特徴量（重要度順）")
print("- 横軸: SHAP値（予測への影響）")
print("- 色: 特徴量の値（赤=高、青=低）")
print("- 各点: 1サンプル")

In [ ]:
# Bar plot: 平均絶対SHAP値
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values[1], test_sample, feature_names=X.columns, plot_type="bar", show=False)
plt.title('SHAP Feature Importance (Mean |SHAP value|)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

# Top 10特徴量をテキストで表示
mean_shap = np.abs(shap_values[1]).mean(axis=0)
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': mean_shap
}).sort_values('importance', ascending=False)

print("\nTop 10 重要な特徴量:")
for i, row in importance_df.head(10).iterrows():
    print(f"{row['feature']:30s}: {row['importance']:.6f}")

### 4.2 個別予測の説明（ローカル解釈）

In [ ]:
# 個別サンプルのSHAP値を可視化（Waterfall plot）
sample_idx = 0

# SHAP値をExplanationオブジェクトに変換
explanation = shap.Explanation(
    values=shap_values[1][sample_idx],
    base_values=explainer.expected_value[1],
    data=test_sample[sample_idx],
    feature_names=X.columns.tolist()
)

plt.figure(figsize=(10, 6))
shap.plots.waterfall(explanation, show=False)
plt.title(f'SHAP Waterfall Plot - Sample {sample_idx}\n(True: {y_test.iloc[sample_idx]}, Pred: {y_pred_nn[sample_idx]})',
         fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Waterfall Plot:")
print("- 基準値（E[f(x)]）から予測値（f(x)）への変化を表示")
print("- 各特徴量がどの程度予測に寄与したかを可視化")
print("- 赤: 予測値を増加させる方向")
print("- 青: 予測値を減少させる方向")

In [ ]:
# 複数サンプルの比較
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for i in range(4):
    sample_idx = i * 10  # 0, 10, 20, 30番目のサンプル
    
    # SHAP値を取得してソート
    shap_vals = shap_values[1][sample_idx]
    top_idx = np.argsort(np.abs(shap_vals))[-10:][::-1]
    
    top_features = X.columns[top_idx]
    top_shap = shap_vals[top_idx]
    
    # プロット
    colors = ['red' if val > 0 else 'blue' for val in top_shap]
    axes[i].barh(range(len(top_shap)), top_shap, color=colors, alpha=0.7, edgecolor='black')
    axes[i].set_yticks(range(len(top_shap)))
    axes[i].set_yticklabels(top_features, fontsize=9)
    axes[i].set_xlabel('SHAP Value', fontsize=10)
    axes[i].set_title(f'Sample {sample_idx} (True: {y_test.iloc[sample_idx]}, Pred: {y_pred_nn[sample_idx]})',
                     fontsize=11, fontweight='bold')
    axes[i].invert_yaxis()
    axes[i].grid(axis='x', alpha=0.3)
    axes[i].axvline(0, color='black', linewidth=0.8)

plt.tight_layout()
plt.show()

print("\n💡 サンプルごとのSHAP値:")
print("- 各サンプルで異なる特徴量が重要")
print("- モデルがどの特徴に注目して予測したかを理解")
print("- 医療診断などで重要な解釈可能性")

## 5. GBDTとの比較

In [ ]:
# LightGBM
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=5,
    random_state=42,
    verbose=-1
)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(20), lgb.log_evaluation(0)]
)

y_pred_lgb = lgb_model.predict(X_test)
y_proba_lgb = lgb_model.predict_proba(X_test)[:, 1]

accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
auc_lgb = roc_auc_score(y_test, y_proba_lgb)

print("=" * 60)
print("LightGBMの性能（テストデータ）")
print("=" * 60)
print(f"Accuracy: {accuracy_lgb:.4f}")
print(f"ROC-AUC:  {auc_lgb:.4f}")

# XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=5,
    random_state=42,
    eval_metric='logloss'
)
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

y_pred_xgb = xgb_model.predict(X_test)
y_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
auc_xgb = roc_auc_score(y_test, y_proba_xgb)

print("\n" + "=" * 60)
print("XGBoostの性能（テストデータ）")
print("=" * 60)
print(f"Accuracy: {accuracy_xgb:.4f}")
print(f"ROC-AUC:  {auc_xgb:.4f}")

In [ ]:
# 性能比較
comparison_df = pd.DataFrame([
    {'Model': 'Neural Network', 'Accuracy': accuracy_nn, 'ROC-AUC': auc_nn},
    {'Model': 'LightGBM', 'Accuracy': accuracy_lgb, 'ROC-AUC': auc_lgb},
    {'Model': 'XGBoost', 'Accuracy': accuracy_xgb, 'ROC-AUC': auc_xgb}
]).sort_values('ROC-AUC', ascending=False)

print("\n" + "=" * 60)
print("モデル性能比較")
print("=" * 60)
print(comparison_df.to_string(index=False))

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
colors = ['steelblue', 'lightgreen', 'coral']
axes[0].bar(comparison_df['Model'], comparison_df['Accuracy'], 
            alpha=0.7, edgecolor='black', color=colors)
axes[0].set_ylabel('Accuracy', fontsize=11)
axes[0].set_title('Accuracy Comparison', fontsize=12, fontweight='bold')
axes[0].set_ylim([0.9, 1.0])
axes[0].grid(axis='y', alpha=0.3)
for i, row in comparison_df.iterrows():
    axes[0].text(row.name, row['Accuracy'], f"{row['Accuracy']:.4f}",
                ha='center', va='bottom', fontweight='bold')

# ROC-AUC
axes[1].bar(comparison_df['Model'], comparison_df['ROC-AUC'], 
            alpha=0.7, edgecolor='black', color=colors)
axes[1].set_ylabel('ROC-AUC', fontsize=11)
axes[1].set_title('ROC-AUC Comparison', fontsize=12, fontweight='bold')
axes[1].set_ylim([0.9, 1.0])
axes[1].grid(axis='y', alpha=0.3)
for i, row in comparison_df.iterrows():
    axes[1].text(row.name, row['ROC-AUC'], f"{row['ROC-AUC']:.4f}",
                ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 観察:")
print("- ニューラルネットワークはGBDTと同等またはそれ以上の性能")
print("- 解釈可能性ではSHAPにより詳細な分析が可能")
print("- 小規模データではGBDTも依然強力")

## 6. 実務でのベストプラクティス

### Tabular Deep Learningを使うべき場面

| 状況 | 推奨 |
|------|------|
| 解釈可能性が重要 | GBDT + SHAP または NN + SHAP |
| データサイズ > 10k | Tabular DL検討 |
| GPUが使える | Tabular DL ✅ |
| 高速推論が必要 | GBDT |
| データサイズ < 5k | GBDT |
| カテゴリカル変数多数 | CatBoost |
| 埋め込み学習が必要 | NN ✅ |

### チェックリスト

```python
# ✅ Tabular NN実装チェックリスト

# 1. データの前処理
# - 特徴量をスケーリングしたか？（StandardScaler推奨）
# - 欠損値を処理したか？
# - カテゴリカル変数をエンコードしたか？

# 2. データ分割
# - 訓練・検証・テストに分割したか？
# - 検証データでEarly stoppingを使うか?

# 3. モデルアーキテクチャ
# - Batch Normalizationを使用したか？
# - Dropoutで過学習を防いだか？
# - 層の深さと幅は適切か？（2-4層推奨）

# 4. 学習
# - Early stoppingを設定したか？
# - 学習率スケジューラを使用したか？
# - 学習曲線を確認したか？
# - 過学習していないか？

# 5. 評価
# - テストデータで性能を評価したか？
# - GBDTと比較したか？
# - SHAPで解釈可能性を確認したか？
```

### ハイパーパラメータチューニングのポイント

1. **層の深さと幅**
   - 深すぎると過学習のリスク
   - 通常2-4層で十分
   - 幅: 64, 128, 256など

2. **Dropout率**
   - 0.2 - 0.5が一般的
   - 過学習の兆候があれば増やす

3. **学習率**
   - Adam: 1e-3 ~ 1e-4
   - スケジューラで動的に調整

4. **バッチサイズ**
   - 小規模データ: 32-64
   - 大規模データ: 128-256

## 7. まとめ

### 本ノートブックで学んだこと

1. **Tabularディープラーニングの必要性**
   - GBDTとの比較
   - 使い分けの基準

2. **PyTorchでのTabular NN実装**
   - Batch Normalization
   - Dropout
   - 適切なアーキテクチャ設計

3. **学習テクニック**
   - Early stopping
   - 学習率スケジューラ
   - 学習曲線の監視

4. **SHAPによる解釈可能性**
   - グローバル特徴量重要度
   - ローカル解釈（個別予測の説明）
   - 様々な可視化手法

5. **GBDTとの比較**
   - 性能の比較
   - それぞれの強み
   - 使い分けの判断基準

### 重要なポイント

- ✅ **スケーリング必須**: ニューラルネットワークは特徴量の正規化が必要
- ✅ **解釈可能性**: SHAPで透明性を確保
- ✅ **Early stopping**: 過学習を防ぐ
- ✅ **GBDTと比較**: まずGBDTを試し、NNで改善を図る
- ✅ **適切なアーキテクチャ**: Batch Norm + Dropout + 2-4層

### 次のステップ

- Notebook 27でKaggle完全ワークフローを学ぶ
- Notebook 28で総合演習プロジェクトに挑戦
- 実際のKaggleコンペでTabular NNを試す
- FT-Transformer、SAINTなど他のTabular DLモデルを学ぶ
- より複雑なアーキテクチャ（ResNet、DenseNet風）を試す